In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
#print(os.listdir("../input"))
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
from numba import jit
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
random_state = 42
np.random.seed(random_state)

@jit is a decorator to mark a function for optimization (http://numba.pydata.org/numba-doc/0.17.0/user/jit.html)

You can use @jit(nopython=True, parallel=True) to optimize the code!

In [2]:
# augment is a function that replicates x and y (appending these replications in the end) according to some criteria 
# Looks like it is trying to augment the data randomly in order increase the power/performance of any model fitted

@jit
def augment(x,y,t=2):
    xs,xn = [],[]
    for i in range(t):
        mask = y>0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xs.append(x1)

    for i in range(t//2):
        mask = y==0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xn.append(x1)

    xs = np.vstack(xs); xn = np.vstack(xn)
    ys = np.ones(xs.shape[0]);yn = np.zeros(xn.shape[0])
    x = np.vstack([x,xs,xn]); y = np.concatenate([y,ys,yn])
    return x,y

In [64]:
for fold, (trn_idx, val_idx) in enumerate(skf.split(train_df, train_df['target'])):
    print(trn_idx)
    
    X_train, y_train = train_df.iloc[trn_idx][features], train_df.iloc[trn_idx]['target']
    X_valid, y_valid = train_df.iloc[val_idx][features], train_df.iloc[val_idx]['target']

[     1      2      3 ... 199996 199997 199999]
[     0      1      2 ... 199997 199998 199999]
[     0      1      2 ... 199996 199998 199999]
[     0      3      4 ... 199996 199997 199998]
[     0      1      2 ... 199997 199998 199999]


In [66]:
trn_idx = [1, 2, 199997]

In [67]:
X_train, y_train = train_df.iloc[trn_idx][features], train_df.iloc[trn_idx]['target']

In [69]:
y_train

1         0
2         0
199997    0
Name: target, dtype: int64

In [70]:
X_train.values

array([[ 1.15006e+01, -4.14730e+00,  1.38588e+01,  5.38900e+00,
         1.23622e+01,  7.04330e+00,  5.62080e+00,  1.65338e+01,
         3.14680e+00,  8.08510e+00, -4.03200e-01,  8.05850e+00,
         1.40239e+01,  8.41350e+00,  5.43450e+00,  1.37003e+01,
         1.38275e+01, -1.55849e+01,  7.80000e+00,  2.85708e+01,
         3.42870e+00,  2.74070e+00,  8.55240e+00,  3.37160e+00,
         6.97790e+00,  1.38910e+01, -1.17684e+01, -2.55860e+00,
         5.04640e+00,  5.48100e-01, -9.29870e+00,  7.87550e+00,
         1.28590e+00,  1.93710e+01,  1.13702e+01,  7.39900e-01,
         2.79950e+00,  5.84340e+00,  1.08160e+01,  3.67830e+00,
        -1.11147e+01,  1.87300e+00,  9.87750e+00,  1.17842e+01,
         1.24440e+00, -4.73797e+01,  7.37180e+00,  1.94800e-01,
         3.44014e+01,  2.57037e+01,  1.18343e+01,  1.32256e+01,
        -4.10830e+00,  6.68850e+00, -8.09460e+00,  1.85995e+01,
         1.93219e+01,  7.01180e+00,  1.92100e+00,  8.86820e+00,
         8.01090e+00, -7.24170e+00,  1.7

In [71]:
y_train.values

array([0, 0, 0], dtype=int64)

In [72]:
X_t, y_t = augment(X_train.values, y_train.values)

In [73]:
X_t

array([[11.5006, -4.1473, 13.8588, ...,  8.7889, 18.356 ,  1.9518],
       [ 8.6093, -2.7457, 12.0805, ...,  8.2675, 14.7222,  0.3965],
       [11.2232, -5.0518, 10.5127, ...,  8.7155, 13.8329,  4.1995],
       [11.5006, -2.7457, 13.8588, ...,  8.7889, 13.8329,  4.1995],
       [11.2232, -5.0518, 10.5127, ...,  8.7155, 14.7222,  0.3965],
       [ 8.6093, -4.1473, 12.0805, ...,  8.2675, 18.356 ,  1.9518]])

In [74]:
len(X_train.values[0])

200

In [60]:
len(X_t[0])

200

Hyperparameters of the model:

In [75]:
params = {
    "objective" : "binary", 
    "metric" : "auc", 
    "boosting": 'gbdt', 
    "max_depth" : -1, "num_leaves" : 13,
    "learning_rate" : 0.01, 
    "bagging_freq": 5, 
    "bagging_fraction" : 0.4, 
    "feature_fraction" : 0.05,
    "min_data_in_leaf": 80, 
    "min_sum_heassian_in_leaf": 10, 
    "tree_learner": "serial", 
    "boost_from_average": "false",
    "bagging_seed" : random_state, 
    "verbosity" : 1, 
    "seed": random_state
}

StratifiedKFold provides train/test indices to split data in train/test sets.

This cross-validation object is a variation of KFold that returns stratified folds. The folds are made by preserving the percentage of samples for each class.

In [78]:
skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = random_state)

oof stand for 'out-of-fold' exaplained in the 16:40 of this video: https://www.youtube.com/watch?v=V5158Oug4W8

oof uses information from **other** fold in the current fold of the cross-validation.
It prevents from using the **actual** target values and, in a certain way, it prevents from overfitting.  

In [86]:
oof = train_df[['ID_code', 'target']]
oof['predict'] = 0
oof.head()

,ID_code,target,predict
0,train_0,0,0
1,train_1,0,0
2,train_2,0,0
3,train_3,0,0
4,train_4,0,0


In [159]:
predictions = test_df[['ID_code']]
predictions.head()

,ID_code
0,test_0
1,test_1
2,test_2
3,test_3
4,test_4


In [92]:
# 'features' is the object 
features = [col for col in train_df.columns if col not in ['target', 'ID_code']]
X_test = test_df[features].values
X_test

array([[ 11.0656,   7.7798,  12.9536, ...,  10.72  ,  15.4722,  -8.7197],
       [  8.5304,   1.2543,  11.3047, ...,   9.8714,  19.1293, -20.976 ],
       [  5.4827, -10.3581,  10.1407, ...,   7.0618,  19.8956, -23.1794],
       ...,
       [ 11.636 ,   2.2769,  11.2074, ...,   9.1933,  11.7905, -22.2762],
       [ 13.5745,  -0.5134,  13.6584, ...,   8.1079,   8.7735,  -0.2122],
       [ 10.4664,   1.807 ,  10.2277, ...,  10.3378,  14.334 ,  -7.7094]])

Now the actual fitting starts. The iteration is under the fold index and the train and test indices. The **skf.split** function generates indices to split data into training and test set.

val_idx is the **VALIDATION** index. That is, the test idx.

In [93]:
for fold, (trn_idx, val_idx) in enumerate(skf.split(train_df, train_df['target'])):
    print(fold)

0
1
2
3
4


In [94]:
for fold, (trn_idx, val_idx) in enumerate(skf.split(train_df, train_df['target'])):
    print(trn_idx)

[     1      2      3 ... 199996 199997 199999]
[     0      1      2 ... 199997 199998 199999]
[     0      1      2 ... 199996 199998 199999]
[     0      3      4 ... 199996 199997 199998]
[     0      1      2 ... 199997 199998 199999]


In [95]:
for fold, (trn_idx, val_idx) in enumerate(skf.split(train_df, train_df['target'])):
    print(val_idx)

[     0     11     12 ... 199988 199992 199998]
[     4     24     32 ... 199993 199994 199996]
[     3      8     15 ... 199979 199980 199997]
[     1      2      6 ... 199983 199991 199999]
[     5     10     16 ... 199989 199990 199995]


### Let's inspect the first iteration of the for loop:

Extracting the first indexes.

In [109]:
first_fold_idx = list(enumerate(skf.split(train_df, train_df['target'])))[0][0]
first_trn_idx  = list(enumerate(skf.split(train_df, train_df['target'])))[0][1][0]
first_val_idx  = list(enumerate(skf.split(train_df, train_df['target'])))[0][1][1]

Extract each mutually exclusive chunk of this iteration.

In [113]:
X_train, y_train = train_df.iloc[first_trn_idx][features], train_df.iloc[first_trn_idx]['target']
X_valid, y_valid = train_df.iloc[first_val_idx][features], train_df.iloc[first_val_idx]['target']

In [115]:
X_train.shape

(159999, 200)

In [116]:
# Changing the original object names: p_valid,yp
N = 3
y_pred_valid, y_pred_test = 0,0

And then the code iterates 3 times over an index that it is not used

In [ ]:
# for i in range(N):

Augmenting the data (**data augmentation**)! https://www.kaggle.com/dansbecker/data-augmentation

In [118]:
X_t, y_t = augment(X_train.values, y_train.values)
X_t = pd.DataFrame(X_t)
X_t = X_t.add_prefix('var_')
X_t.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
1,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
2,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
3,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104
4,11.4763,-2.3182,12.6080,8.6264,10.9621,3.5609,4.5322,15.2255,3.5855,5.9790,...,-6.3068,6.6025,5.2912,0.4403,14.9452,1.0314,-3.6241,9.7670,12.5809,-4.7602


In [119]:
X_t.shape

(336076, 200)

This is the most important part of the script (this takes some time to run). This is where you train the actual model with the augmented dataset previously built with two specific Dataset class of lightGBM

You can type `help(lgb.train)` for more details.

**Note that the train_data is inside the valid_sets... This is to calculate the AUC metric for either training data and validation (test) data. The result is going to be printed for the training's auc and valid_1's auc**

In [123]:
trn_data = lgb.Dataset(X_t, label=y_t)
val_data = lgb.Dataset(X_valid, label=y_valid)
evals_result = {}
lgb_clf = lgb.train(params,
                    trn_data,
                    num_boost_round = 100000,
                    valid_sets = [trn_data, val_data], # List of data to be evaluated on during training.
                    early_stopping_rounds=3000,
                    verbose_eval = 5000,
                    evals_result=evals_result)

Training until validation scores don't improve for 3000 rounds.
[5000]	training's auc: 0.917161	valid_1's auc: 0.897945
[10000]	training's auc: 0.929624	valid_1's auc: 0.900436
[15000]	training's auc: 0.939603	valid_1's auc: 0.900341
Early stopping, best iteration is:
[12147]	training's auc: 0.934067	valid_1's auc: 0.900487


Note that the model estimated is the last one that it is presented under *''best iteration is:''*.

Now, we can predict the probability of the outcome as following:

In [127]:
# In the code this is += instead of += 

y_pred_valid = lgb_clf.predict(X_valid) # In the code this is += 
y_pred_valid

array([0.01039275, 0.07985715, 0.02033983, ..., 0.00455992, 0.03712945,
       0.07780325])

... also for the test data:

In [128]:
y_pred_test = lgb_clf.predict(X_test) # In the code this is +=
y_pred_test

array([0.15927169, 0.34433102, 0.23573733, ..., 0.00474793, 0.15390632,
       0.08317883])

IN THE CODE, THE PREDICTED PROBABILITIES ARE BEING SUMMED with the += OPERATOR N TIMES! AND THEN, LATER, THESE PREDICTIONS ARE DIVIDED BY N. THEREFORE, THEY CALCULATED SOME KIND OF AVERAGE OF THE PREDICTED PROBABILITES!

Now, after building tre predicted probability vector (which is the sum of N values), it is time to add these values into the oof object (which was previously built with train data).

In [150]:
oof['predict'][first_val_idx] = y_pred_valid / 1 # In the code, the denominator is N.
oof.head()

,ID_code,target,predict
0,train_0,0,0.020786
1,train_1,0,0.000000
2,train_2,0,0.000000
3,train_3,0,0.000000
4,train_4,0,0.000000


In the code, it is just for exploration some summary statistics of all auc's among the folds with `val_aucs.append(val_score)` and then checking mean and std of the `val_aucs` object.

In [ ]:
#val_score = roc_auc_score(y_valid, y_pred_valid)
#val_score

Now we built the predicted probabilities for each of the folds. Later, we use each of the values to take the mean of these probabilities to submit.

In [161]:
predictions['fold{}'.format(first_fold_idx+1)] = y_pred_test / 1 # The denominator is N in the code
predictions.head()

,ID_code,fold1
0,test_0,0.159272
1,test_1,0.344331
2,test_2,0.235737
3,test_3,0.275450
4,test_4,0.058301


The submissions is a pandas DF with the mean of the probabilities of all folds.

In [163]:
predictions['target'] = np.mean(predictions[[col for col in predictions.columns if col not in ['ID_code', 'target']]].values, axis=1)
submission = pd.DataFrame({"ID_code":test_df["ID_code"].values})
submission["target"] = predictions['target']
submission.head()

,ID_code,target
0,test_0,0.159272
1,test_1,0.344331
2,test_2,0.235737
3,test_3,0.275450
4,test_4,0.058301
